In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [154]:
print(len(metai))
print(len(kebulas))
print(len(kaina))
print(len(kuras))
print(len(pavaros))
print(len(galingumas))
print(len(rida))
print(len(nuorodos))
print(len(gamintojas))
print(len(modelis))
print(len(pirmaregistracija))

200
200
200
200
200
200
200
200
200
200
200


In [ ]:
print(metai)
print(kebulas)
print(kaina)
print(kuras)
print(pavaros)
print(galingumas)
print(rida)
print(nuorodos)
print(gamintojas)
print(modelis)
print(pirmaregistracija)
print(technikine)

In [164]:
SDB = sqlite3.connect('autoplius4.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists T1
(
gamintojas text,
modelis text,
kaina integer,
metai integer,
kebulas text,
kuras text,
pavaros text,
galingumas text,
rida integer,
nuorodos text,
pirmaregistracija text,
technikine text

)
'''
Cs.execute(sql)

sql_template = '''insert into T1 values (?,?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
for a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12 in zip(gamintojas, modelis, kaina, metai, kebulas, kuras, pavaros, galingumas, rida, nuorodos, pirmaregistracija, technikine):
    Cs.execute(sql_template, (a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12))

SDB.commit() # commit() būtinas, jei norite įrašyti duoenis į DB

SDB.close()

In [153]:
opcijos = Options()
opcijos.add_argument('--incognito')

modelis,gamintojas,metai,kebulas,kaina,kuras,pavaros,galingumas,rida,regsalis,nuorodos,pirmaregistracija=[],[],[],[],[],[],[],[],[],[],[],[]
technikine=[]


driver = webdriver.Chrome(options=opcijos)

for pgnr in range(1, 11):
    url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr='+str(pgnr) 
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    automobiliai = bs.find_all('a', {'class':'announcement-item'})

    for auto in automobiliai:
        kuras.append(None)
        metai.append(None)
        rida.append(None)
        galingumas.append(None)
        kebulas.append(None)
        pavaros.append(None)
        technikine.append(None)
        pirmaregistracija.append(None)
        
        a=auto.find('div', {'class':'announcement-parameters-block'}).find_all('span')
        url1 = auto['href'] 
        driver.get(url1)
        time.sleep(2)
        source1 = driver.page_source
        bs1 = BeautifulSoup(source1, 'html.parser')
        
        for duomenys in bs1.find_all('div', {'class':'parameter-row'}):
            try:
                duomenys.find('div',{'class':'parameter-label'}).get_text().strip()
                duomenys.find('div',{'class':'parameter-value'}).get_text().strip()
            except:
                pass
            else:     
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kuro tipas':
                    kuras[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirma registracija':
                    metai[-1]=int(duomenys.find('div',{'class':'parameter-value'}).get_text().strip()[:4])
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Rida':
                    rida[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().replace(' km', '').replace(' ',''))
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Variklis':
                    galia=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip().split()[-1])
                    if str(galia[-3:-1]) == 'kW':
                        galingumas[-1]=((galia[:-3].replace('(', '')))
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Kėbulo tipas':
                    kebulas[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pavarų dėžė':
                    pavaros[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Tech. apžiūra iki':
                    technikine[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
                if duomenys.find('div',{'class':'parameter-label'}).get_text().strip() == 'Pirmosios registracijos šalis':
                    pirmaregistracija[-1]=(duomenys.find('div',{'class':'parameter-value'}).get_text().strip())
        
        try:
            bs1.find_all('li', {'class':'crumb'})[2].get_text().strip()
        except:
            gamintojas.append(None)
            pass
        else:
            gamintojas.append(bs1.find_all('li', {'class':'crumb'})[2].get_text().strip())
            
        try:
            bs1.find_all('li', {'class':'crumb'})[3].get_text().strip()
        except:
            modelis.append(None)
            pass
        else:
            modelis.append(bs1.find_all('li', {'class':'crumb'})[3].get_text().strip())
            
        kaina.append(int(auto.find('div', {'class':'announcement-pricing-info'}).find('strong').get_text().strip().replace(' ','').replace('€', '')))
        nuorodos.append(auto['href'])
            
            
driver.close()